In [1]:
# import pandas + plotly theme
import pandas as pd
from utils.plotly_theme import *

In [2]:
# test interpolation function (for cap,cop,plr)
from utils.interp import interp_vector

fp = [3.815, 3.955, 4.133, 3.74] # cop
xp = [1_465_000, 1_098_750, 732_500, 366_250] # cap

fp_1 = [3.74, 4.133, 3.955, 3.815] # cop
xp_1 = [366_250, 732_500, 1_098_750, 1_465_000] # cap

x = [1_200_000] # 1_465_200

print(interp_vector(xp, fp, x))
print(interp_vector(xp_1, fp_1, x))

[3.74]
[3.91629693]


In [3]:
# Example usage of the decarb-tool

from src.metadata import Metadata
from src.emissions import EmissionScenario
from utils.inputs import get_city_and_zone

# Load location data
location_data = pd.read_csv("data/input/locations.csv")
zip_code = "94720" # Berkeley, CA -> will be replaced by user input from UI (search by zip-code or city name)
locations = get_city_and_zone(zip_code, location_data)

# Based on zip-code, SET location, climate_zone, time_zone
location = locations.city
ashrae_climate_zone = locations.ashrae_climate_zone
time_zone = locations.time_zone
# gea_grid_region = locations.gea_grid_region # not yet implemented

# Set building type, building vintage, load type and equipment via equipment scenario -> eventually replaced by user input from UI
building_type = "OfficeLarge" # currently we have "Hospital", "OfficeLarge", "OutPatientHealthCare"
vintage = 2004 # currently we have 2004, 2022
load_type = "load_simulated"
equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"] # list of equipment scenarios to compare

settings = Metadata.create(
    location=location,
    ashrae_climate_zone=ashrae_climate_zone,
    time_zone=time_zone,
    building_type=building_type,
    vintage=vintage,
    load_type=load_type,
    equipment_scenarios=equipment_scenarios,
)

In [4]:
# check it emission scenarios are loaded correctly

scenario = settings.get_emission_scenario("em_scenario_a")
print(scenario.year)

settings.model_dump()

2025


{'location': 'Berkeley',
 'building_type': 'OfficeLarge',
 'vintage': 2004,
 'load_type': 'load_simulated',
 'ashrae_climate_zone': '3C',
 'equipment_scenarios': ['eq_scenario_1',
  'eq_scenario_2',
  'eq_scenario_3',
  'eq_scenario_4',
  'eq_scenario_5'],
 'emission_settings': [{'em_scen_id': 'em_scenario_a',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Includes pre-combustion',
   'shortrun_weighting': 0.0,
   'annual_refrig_leakage_percent': 0.05,
   'annual_ng_leakage_g_per_kWh': 239.2,
   'year': 2025},
  {'em_scen_id': 'em_scenario_b',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Includes pre-combustion',
   'shortrun_weighting': 0.0,
   'annual_refrig_leakage_percent': 0.05,
   'annual_ng_leakage_g_per_kWh': 239.2,
   'year': 2035},
  {'em_scen_id': 'em_scenario_c',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',


In [5]:
# test equipment library load

from src.equipment import load_library
equipment_library = load_library("data/input/equipment_data.json").model_dump()

equipment_library

{'equipment': [{'eq_id': 'hr01',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water',
   'eq_manufacturer': None,
   'model': 'GenericHR_WWHP',
   'fuel': 'electricity',
   'refrigerant': 'R-454B',
   'refrigerant_weight_g': 200000.0,
   'refrigerant_gwp': 466.0,
   'capacity_W': None,
   'performance': {'heating': {'cop_curve': None,
     'cap_curve': None,
     'plr_curve': {'capacity_W': [135700.0,
       271300.0,
       319100.0,
       606700.0,
       414900.0,
       510700.0],
      'cop': [2.28, 2.28, 2.3, 2.33, 2.35, 2.37]},
     'efficiency': None,
     'constraints': None}},
   'emissions': None},
  {'eq_id': 'hr02',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water_scroll',
   'eq_manufacturer': None,
   'model': 'IMC-WWHP-30Ton-120HHWST',
   'fuel': 'electricity',
   'refrigerant': 'R-454B',
   'refrigerant_weight_g': 17700.0,
   'refrigerant_gwp': 466.0,
   'capacity_W': None,
   'performance': {'heating': {'cop_curve': None,
     'cap_curve': N

In [6]:
# check library in normalized dataframe format

equipment_data = pd.json_normalize(equipment_library["equipment"])
equipment_data.head(50)

,eq_id,eq_type,eq_subtype,eq_manufacturer,model,fuel,refrigerant,refrigerant_weight_g,refrigerant_gwp,capacity_W,...,performance.cooling.cop_curve.t_out_C,performance.cooling.cop_curve.cop,performance.cooling.cap_curve.t_out_C,performance.cooling.cap_curve.capacity_W,performance.cooling.plr_curve,performance.cooling.efficiency,performance.cooling.constraints,performance.cooling.cap_curve,emissions.co2_kg_per_mwh,performance.cooling.cop_curve
0,hr01,hr_heat_pump,water_to_water,None,GenericHR_WWHP,electricity,R-454B,200000.0,466.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hr02,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-120HHWST,electricity,R-454B,17700.0,466.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hr03,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-140HHWST,electricity,R-454B,17700.0,466.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hr04,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-120HHWST,electricity,R-454B,29000.0,466.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hr05,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-140HHWST,electricity,R-454B,29000.0,466.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,hr06,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-120HHWST,electricity,R-515B,300800.0,299.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,hr07,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-140HHWST,electricity,R-515B,300800.0,299.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,hr08,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-120HHWST,electricity,R-515B,453600.0,299.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,hr09,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-140HHWST,electricity,R-515B,453600.0,299.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,hp01,heat_pump,air_to_water,None,GenericAWHP,electricity,R-454B,54000.0,466.0,NaN,...,"[15.0, 25.0, 35.0, 45.0]","[4.4, 4.0, 2.97, 2.65]","[15.0, 25.0, 35.0, 45.0]","[250000.0, 220000.0, 201200.0, 183000.0]",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# test adding a new emission scenario

settings.add_emission_scenario(
    EmissionScenario(
        em_scen_id="new_007",
        grid_scenario="MidCase",
        gea_grid_region="CAISO",
        time_zone="America/New_York",
        emission_type="Combustion only",
        annual_ng_leakage_g_per_kWh=2.0,
        annual_refrig_leakage_percent=5.0,
        shortrun_weighting=1.0,
        year=2025,
    )
)

print(settings["new_007"].year)
print(settings["new_007"].grid_scenario)

x = settings.__getitem__("new_007")
print(x.gea_grid_region)


2025
MidCase
CAISO


In [8]:

# check if emission scenario was added correctly
from pprint import pprint

pprint(settings.dict())

{'ashrae_climate_zone': '3C',
 'building_type': 'OfficeLarge',
 'emission_settings': [{'annual_ng_leakage_g_per_kWh': 239.2,
                        'annual_refrig_leakage_percent': 0.05,
                        'em_scen_id': 'em_scenario_a',
                        'emission_type': 'Includes pre-combustion',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 0.0,
                        'time_zone': 'America/Los_Angeles',
                        'year': 2025},
                       {'annual_ng_leakage_g_per_kWh': 239.2,
                        'annual_refrig_leakage_percent': 0.05,
                        'em_scen_id': 'em_scenario_b',
                        'emission_type': 'Includes pre-combustion',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 0.0,
                        'time_zo

/var/folders/x4/3lxftn1s4zgf32y4j4tqx4kh0000gn/T/ipykernel_94922/3253440217.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(settings.dict())


In [9]:
# Get datasets based on input (location, load_type) saved in settings object
from src.loads import get_load_data
from src.emissions import get_emissions_data

# get load data
load_data = get_load_data(settings)

# get emission data
emissions_data = get_emissions_data(settings["em_scenario_a"])

# inspect load data
load_data.df.head(30)

/Users/toby/Desktop/Repositories/decarb-tool/src/loads.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=False)


,t_out_C,heating_W,cooling_W
timestamp,,,
2001-01-01 01:00:00,8.000000,9144.992218,0.000000
2001-01-01 02:00:00,8.000000,7880.200257,0.000000
2001-01-01 03:00:00,8.583333,6636.406252,0.000000
2001-01-01 04:00:00,9.000000,5463.693419,0.000000
2001-01-01 05:00:00,9.000000,7712.880953,0.000000
2001-01-01 06:00:00,8.416667,8015.533175,0.000000
2001-01-01 07:00:00,8.583333,10638.064920,0.000000
2001-01-01 08:00:00,9.583333,2872.637738,0.076299
2001-01-01 09:00:00,11.166667,0.000000,0.141181


In [10]:
# inspect emissions data
emissions_data.df.head(30)

,em_scen_id,emission_scenario,gea_grid_region,time_zone,emission_type,year,lrmer_co2e_c,lrmer_co2e_p,lrmer_co2e,srmer_co2e_c,srmer_co2e_p,srmer_co2e
timestamp,,,,,,,,,,,,
2025-01-01 00:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,408.0,137.7,545.8,561.2,173.2,734.4
2025-01-01 01:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,431.1,142.3,573.4,547.6,163.8,711.5
2025-01-01 02:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,429.1,147.9,577.0,461.2,162.2,623.4
2025-01-01 03:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,413.6,149.6,563.2,480.7,167.0,647.7
2025-01-01 04:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,417.8,145.3,563.0,480.9,179.3,660.3
2025-01-01 05:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,396.4,137.6,534.0,533.5,174.0,707.5
2025-01-01 06:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,364.3,126.7,490.9,546.2,177.9,724.1
2025-01-01 07:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,269.0,91.7,360.7,656.7,154.8,811.5
2025-01-01 08:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Includes pre-combustion,2025,139.6,46.7,186.2,554.9,158.2,713.0


In [11]:
# Test-loading individual pieces from equipment library (not changing the actual sim settings)

from src.equipment import load_library

library = load_library("data/input/equipment_data.json")

hp_a = library.get_equipment("hp01")
print(hp_a)

scenario_a = library.get_scenario("eq_scenario_1")
print(scenario_a)

eq_id='hp01' eq_type='heat_pump' eq_subtype='air_to_water' eq_manufacturer=None model='GenericAWHP' fuel='electricity' refrigerant='R-454B' refrigerant_weight_g=54000.0 refrigerant_gwp=466.0 capacity_W=None performance={'heating': Performance(cop_curve=COPCurve(t_out_C=[-12.2, -6.7, -1.1, 4.4, 10.0, 20.0, 30.0], cop=[1.98, 2.06, 2.32, 2.7, 3.0, 3.2, 3.5]), cap_curve=CapCurve(t_out_C=[-12.2, -6.7, -1.1, 4.4, 10.0, 20.0, 30.0], capacity_W=[132000.0, 154200.0, 168600.0, 195500.0, 229200.0, 254000.0, 272000.0]), plr_curve=None, efficiency=None, constraints=None), 'cooling': Performance(cop_curve=COPCurve(t_out_C=[15.0, 25.0, 35.0, 45.0], cop=[4.4, 4.0, 2.97, 2.65]), cap_curve=CapCurve(t_out_C=[15.0, 25.0, 35.0, 45.0], capacity_W=[250000.0, 220000.0, 201200.0, 183000.0]), plr_curve=None, efficiency=None, constraints=None)} emissions=None
eq_scen_id='eq_scenario_1' eq_scen_name='Gas Boiler+AC Chiller' hr_wwhp=None awhp=None awhp_sizing_mode='peak_load_percentage' awhp_sizing_value=1.0 awhp_u

In [12]:
# calculate site energy for all equipment scenarios in settings object

from src.energy import loads_to_site_energy

site_energy = loads_to_site_energy(load_data, library, settings.equipment_scenarios, detail=True)

print(f"Equipment scenarios considered: {len(settings.equipment_scenarios)}")
print(f"Computed number of years: {len(site_energy) / 8760}")

site_energy.head(30)


Equipment scenarios considered: 5
Computed number of years: 5.0


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,eq_scen_name,awhp_refrigerant,total_awhp_refrigerant_weight_kg,total_awhp_refrigerant_gwp_kg,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,hr_wwhp_refrigerant,hr_wwhp_refrigerant_weight_kg,hr_wwhp_refrigerant_gwp_kg
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,8.0000,9144.9922,0.0000,0.0000,11431.2403,9144.9922,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 02:00:00,8.0000,7880.2003,0.0000,0.0000,9850.2503,7880.2003,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 03:00:00,8.5833,6636.4063,0.0000,0.0000,8295.5078,6636.4063,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 04:00:00,9.0000,5463.6934,0.0000,0.0000,6829.6168,5463.6934,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 05:00:00,9.0000,7712.8810,0.0000,0.0000,9641.1012,7712.8810,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 06:00:00,8.4167,8015.5332,0.0000,0.0000,10019.4165,8015.5332,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 07:00:00,8.5833,10638.0649,0.0000,0.0000,13297.5812,10638.0649,0.0000,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 08:00:00,9.5833,2872.6377,0.0763,0.0269,3590.7972,2872.6377,0.0763,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-01 09:00:00,11.1667,0.0000,0.1412,0.0497,0.0000,0.0000,0.1412,NaN,NaN,NaN,...,Gas Boiler+AC Chiller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# check available emission scenarios before running site to source
settings.list_emission_scenarios()

['em_scenario_a', 'em_scenario_b', 'em_scenario_c', 'new_007']

In [14]:
# run site to source

from src.energy import site_to_source

source_energy = site_to_source(site_energy, metadata=settings)

print(f"Equipment scenarios considered: {len(settings.equipment_scenarios)}")
print(f"Emission scenarios considered: {len(settings.emission_settings)}")
print(f"Computed number of years: {len(source_energy) / 8760}")

source_energy.to_csv("debug_source_energy.csv", index=False)

source_energy.head(30)

Equipment scenarios considered: 5
Emission scenarios considered: 4
Computed number of years: 20.0


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,srmer_co2e_p,srmer_co2e,shortrun_weighting,year,elec_emissions,gas_emissions,total_refrig_gwp_kg,total_refrig_emissions,total_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,8.0000,9144.9922,0.0000,0.0000,11431.2403,9144.9922,0.0000,NaN,NaN,NaN,...,163.8,711.5,0.0,2025,0.000000,2.734353,21.6189,1.080945,3.815298,em_scenario_a
2025-01-01 02:00:00,8.0000,7880.2003,0.0000,0.0000,9850.2503,7880.2003,0.0000,NaN,NaN,NaN,...,162.2,623.4,0.0,2025,0.000000,2.356180,21.6189,1.080945,3.437125,em_scenario_a
2025-01-01 03:00:00,8.5833,6636.4063,0.0000,0.0000,8295.5078,6636.4063,0.0000,NaN,NaN,NaN,...,167.0,647.7,0.0,2025,0.000000,1.984285,21.6189,1.080945,3.065230,em_scenario_a
2025-01-01 04:00:00,9.0000,5463.6934,0.0000,0.0000,6829.6168,5463.6934,0.0000,NaN,NaN,NaN,...,179.3,660.3,0.0,2025,0.000000,1.633644,21.6189,1.080945,2.714589,em_scenario_a
2025-01-01 05:00:00,9.0000,7712.8810,0.0000,0.0000,9641.1012,7712.8810,0.0000,NaN,NaN,NaN,...,174.0,707.5,0.0,2025,0.000000,2.306151,21.6189,1.080945,3.387096,em_scenario_a
2025-01-01 06:00:00,8.4167,8015.5332,0.0000,0.0000,10019.4165,8015.5332,0.0000,NaN,NaN,NaN,...,177.9,724.1,0.0,2025,0.000000,2.396644,21.6189,1.080945,3.477589,em_scenario_a
2025-01-01 07:00:00,8.5833,10638.0649,0.0000,0.0000,13297.5812,10638.0649,0.0000,NaN,NaN,NaN,...,154.8,811.5,0.0,2025,0.000000,3.180781,21.6189,1.080945,4.261726,em_scenario_a
2025-01-01 08:00:00,9.5833,2872.6377,0.0763,0.0269,3590.7972,2872.6377,0.0763,NaN,NaN,NaN,...,158.2,713.0,0.0,2025,0.000005,0.858919,21.6189,1.080945,1.939869,em_scenario_a
2025-01-01 09:00:00,11.1667,0.0000,0.1412,0.0497,0.0000,0.0000,0.1412,NaN,NaN,NaN,...,164.6,623.0,0.0,2025,0.000008,0.000000,21.6189,1.080945,1.080953,em_scenario_a


In [15]:
df = source_energy.copy()

df

,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,srmer_co2e_p,srmer_co2e,shortrun_weighting,year,elec_emissions,gas_emissions,total_refrig_gwp_kg,total_refrig_emissions,total_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,8.0000,9144.9922,0.000,0.0000,11431.2403,9144.9922,0.000,NaN,NaN,NaN,...,163.8,711.5,0.0,2025,0.000000,2.734353,21.6189,1.080945,3.815298,em_scenario_a
2025-01-01 02:00:00,8.0000,7880.2003,0.000,0.0000,9850.2503,7880.2003,0.000,NaN,NaN,NaN,...,162.2,623.4,0.0,2025,0.000000,2.356180,21.6189,1.080945,3.437125,em_scenario_a
2025-01-01 03:00:00,8.5833,6636.4063,0.000,0.0000,8295.5078,6636.4063,0.000,NaN,NaN,NaN,...,167.0,647.7,0.0,2025,0.000000,1.984285,21.6189,1.080945,3.065230,em_scenario_a
2025-01-01 04:00:00,9.0000,5463.6934,0.000,0.0000,6829.6168,5463.6934,0.000,NaN,NaN,NaN,...,179.3,660.3,0.0,2025,0.000000,1.633644,21.6189,1.080945,2.714589,em_scenario_a
2025-01-01 05:00:00,9.0000,7712.8810,0.000,0.0000,9641.1012,7712.8810,0.000,NaN,NaN,NaN,...,174.0,707.5,0.0,2025,0.000000,2.306151,21.6189,1.080945,3.387096,em_scenario_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 20:00:00,10.2500,157132.5877,0.000,52290.3786,0.0000,157132.5877,0.000,0.0,0.0,2.33,...,198.4,789.2,1.0,2025,30.898385,0.000000,32.2697,161.348500,192.246885,new_007
2025-12-31 21:00:00,10.7500,183872.3932,0.000,60985.8684,0.0000,183872.3932,0.000,0.0,0.0,2.33,...,202.1,742.8,1.0,2025,32.975059,0.000000,32.2697,161.348500,194.323559,new_007
2025-12-31 22:00:00,12.0000,190518.3454,0.000,62670.5084,0.0000,190518.3454,0.000,0.0,0.0,2.33,...,192.7,767.1,1.0,2025,36.004207,0.000000,32.2697,161.348500,197.352707,new_007


In [16]:
from src.visuals import plot_energy_and_emissions

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"]
emission_scenarios = ["em_scenario_a"]

plot_energy_and_emissions(df, equipment_scenarios, emission_scenarios, unit_mode="SI")

In [17]:
from src.visuals import plot_emission_scenarios_grouped

df = source_energy.copy()

emission_scenarios = ['em_scenario_a', 'em_scenario_b', 'em_scenario_c']

plot_emission_scenarios_grouped(df, equipment_scenarios, emission_scenarios, unit_mode="SI")

In [18]:
from src.visuals import plot_meter_timeseries

df = source_energy.copy()

equipment_scenarios = "eq_scenario_4"
emission_scenarios = "em_scenario_a"

plot_meter_timeseries(df, equipment_scenario=equipment_scenarios, emission_scenario=emission_scenarios, stacked=True, include_gas=True, unit_mode="IP")

In [19]:
from src.visuals import plot_emissions_heatmap

df = source_energy.copy()

equipment_scenarios = "eq_scenario_1"
emission_scenarios = "em_scenario_a"

plot_emissions_heatmap(df, equipment_scenarios, emission_scenarios, unit_mode="SI", emission_type="total_emissions")

In [20]:
from src.visuals import plot_scatter_temp_vs_variable

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"]
emission_scenarios = ["em_scenario_a"]

plot_scatter_temp_vs_variable(
    df,
    y_var="total_emissions",
    equipment_scenarios=equipment_scenarios,
    emission_scenarios=emission_scenarios,
    unit_mode="IP",
)

In [21]:
#! Update for mutliple scenarios and years

from src.visuals import plot_energy_breakdown

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1"]
emission_scenarios = ["em_scenario_a"]

plot_energy_breakdown(df, equipment_scenarios, emission_scenarios)


In [22]:
df = source_energy.copy()

df.columns



Index(['t_out_C', 'heating_W', 'cooling_W', 'elec_Wh', 'gas_Wh', 'hhw_W',
       'chw_W', 'hr_hhw_W', 'hr_chw_W', 'hr_cop_h', 'elec_hr_Wh', 'awhp_num_h',
       'awhp_cap_h_W', 'awhp_cop_h', 'awhp_hhw_W', 'elec_awhp_h_Wh',
       'boiler_eff', 'boiler_hhw_W', 'gas_boiler_Wh', 'res_hhw_W',
       'elec_res_Wh', 'awhp_num_c', 'awhp_cap_c_W', 'awhp_cop_c', 'awhp_chw_W',
       'elec_awhp_c_Wh', 'chiller_cop', 'chiller_chw_W', 'elec_chiller_Wh',
       'chiller_refrigerant', 'chiller_refrigerant_weight_kg',
       'chiller_refrigerant_gwp_kg', 'scenario_id', 'eq_scen_name',
       'awhp_refrigerant', 'total_awhp_refrigerant_weight_kg',
       'total_awhp_refrigerant_gwp_kg', 'max_cap_h_hr_W', 'min_cap_h_hr_W',
       'simult_h_hr_W', 'hr_wwhp_refrigerant', 'hr_wwhp_refrigerant_weight_kg',
       'hr_wwhp_refrigerant_gwp_kg', 'elec_emissions_rate', 'lrmer_co2e_c',
       'lrmer_co2e_p', 'lrmer_co2e', 'srmer_co2e_c', 'srmer_co2e_p',
       'srmer_co2e', 'shortrun_weighting', 'year', 'elec_em

In [23]:
df.head(10)

,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,srmer_co2e_p,srmer_co2e,shortrun_weighting,year,elec_emissions,gas_emissions,total_refrig_gwp_kg,total_refrig_emissions,total_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,8.0000,9144.9922,0.0000,0.0000,11431.2403,9144.9922,0.0000,NaN,NaN,NaN,...,163.8,711.5,0.0,2025,0.000000,2.734353,21.6189,1.080945,3.815298,em_scenario_a
2025-01-01 02:00:00,8.0000,7880.2003,0.0000,0.0000,9850.2503,7880.2003,0.0000,NaN,NaN,NaN,...,162.2,623.4,0.0,2025,0.000000,2.356180,21.6189,1.080945,3.437125,em_scenario_a
2025-01-01 03:00:00,8.5833,6636.4063,0.0000,0.0000,8295.5078,6636.4063,0.0000,NaN,NaN,NaN,...,167.0,647.7,0.0,2025,0.000000,1.984285,21.6189,1.080945,3.065230,em_scenario_a
2025-01-01 04:00:00,9.0000,5463.6934,0.0000,0.0000,6829.6168,5463.6934,0.0000,NaN,NaN,NaN,...,179.3,660.3,0.0,2025,0.000000,1.633644,21.6189,1.080945,2.714589,em_scenario_a
2025-01-01 05:00:00,9.0000,7712.8810,0.0000,0.0000,9641.1012,7712.8810,0.0000,NaN,NaN,NaN,...,174.0,707.5,0.0,2025,0.000000,2.306151,21.6189,1.080945,3.387096,em_scenario_a
2025-01-01 06:00:00,8.4167,8015.5332,0.0000,0.0000,10019.4165,8015.5332,0.0000,NaN,NaN,NaN,...,177.9,724.1,0.0,2025,0.000000,2.396644,21.6189,1.080945,3.477589,em_scenario_a
2025-01-01 07:00:00,8.5833,10638.0649,0.0000,0.0000,13297.5812,10638.0649,0.0000,NaN,NaN,NaN,...,154.8,811.5,0.0,2025,0.000000,3.180781,21.6189,1.080945,4.261726,em_scenario_a
2025-01-01 08:00:00,9.5833,2872.6377,0.0763,0.0269,3590.7972,2872.6377,0.0763,NaN,NaN,NaN,...,158.2,713.0,0.0,2025,0.000005,0.858919,21.6189,1.080945,1.939869,em_scenario_a
2025-01-01 09:00:00,11.1667,0.0000,0.1412,0.0497,0.0000,0.0000,0.1412,NaN,NaN,NaN,...,164.6,623.0,0.0,2025,0.000008,0.000000,21.6189,1.080945,1.080953,em_scenario_a


In [24]:
lrmer_co2e_c = 100
lrmer_co2e_p = 50
srmer_co2e_c = 200
srmer_co2e_p = 100
shortrun_weighting = 0

elec_emissions_rate = (
    (lrmer_co2e_c + lrmer_co2e_p)
    * (1 - shortrun_weighting)
) + (
    (srmer_co2e_c + srmer_co2e_p)
    * shortrun_weighting
)

print(elec_emissions_rate)

150
